# IMPUTACIÓN DE MISSINGS EN VARIABLES CATEGÓRICAS

Implementación de la técnica de imputación univariante mediante la moda para variables categóricas.

In [1]:
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as p
from functools import reduce
from pyspark import keyword_only
from pyspark.ml.param.shared import *
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.pipeline import Estimator, Model, Pipeline

In [2]:
spark = SparkSession.builder.appName("Predict Fraud").master("local").getOrCreate()

22/06/23 00:22:16 WARN Utils: Your hostname, lu-Aspire-A515-55 resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/06/23 00:22:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/23 00:22:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/23 00:22:17 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: ed:2e:48:c8:ae:53:5d:45
22/06/23 00:22:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/23 00:22:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/23 00:22:17 WARN Utils: Service 'SparkUI' could no

In [3]:
df = spark.read.csv('../../Datasets/Missings_variables_categóricas.csv',header=True)

In [4]:
df.show()

+----------+----------+----------+
|Variable_1|Variable_2|Variable_3|
+----------+----------+----------+
|      Gato|      null|     Perro|
|     Perro|     Perro|     Perro|
|     Perro|    Conejo|     Perro|
|      Gato|    Conejo|     Perro|
|    Conejo|     Perro|     Perro|
+----------+----------+----------+



Vemos que tenemos un missing en una de nuestras columnas. A continuación vamos a implementar estimador y un transformer que encuentren e imputen los missings en variables categóricas usando la moda. Pero si una variable tiene más de un cierto porcentaje de sus valores como missings la eliminaremos. 

Vamos a realizar el procedimiento con la estructura típica de MLlib para luego poder aprovechar este código para meterlo en una Pipeline con todo el preprocesamiento para cuando realizemos un flujo de preprocesado completo.

In [5]:
# Clases que inicializan los distintos parámetros del algoritmo
class HasColumnasMissingsImputar(Params):
    def __init__(self):
        super(HasColumnasMissingsImputar, self).__init__()
        self.columnasMissingsImputar = Param(self, "columnasMissingsImputar", "columnasMissingsImputar", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasMissingsImputar(self, value):
        return self.set(self.columnasMissingsImputar, value)

    def getColumnasMissingsImputar(self):
        return self.getOrDefault(self.columnasMissingsImputar)

class HasColumnasMissingsEliminar(Params):
    def __init__(self):
        super(HasColumnasMissingsEliminar, self).__init__()
        self.columnasMissingsEliminar = Param(self, "columnasMissingsEliminar", "columnasMissingsEliminar", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasMissingsEliminar(self, value):
        return self.set(self.columnasMissingsEliminar, value)

    def getColumnasMissingsEliminar(self):
        return self.getOrDefault(self.columnasMissingsEliminar)
    
class HasListaModas(Params):
    def __init__(self):
        super(HasListaModas, self).__init__()
        self.listaModas = Param(self, "listaModas", "listaModas", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaModas(self, value):
        return self.set(self.listaModas, value)

    def getListaModas(self):
        return self.getOrDefault(self.listaModas)
    
class HasPorcentajeMissings(Params):
    def __init__(self):
        super(HasPorcentajeMissings, self).__init__()
        self.porcentajeMissings = Param(self, "porcentajeMissings", "porcentajeMissings", 
                                        typeConverter=TypeConverters.identity)
        
    def setPorcentajeMissings(self, value):
        return self.set(self.porcentajeMissings, value)

    def getPorcentajeMissings(self):
        return self.getOrDefault(self.porcentajeMissings)
    

# Clase que define el estimador
class ImputaciónMissingsEstimator(Estimator, HasPorcentajeMissings, DefaultParamsReadable, DefaultParamsWritable):
    
    @keyword_only
    def __init__(self, porcentajeMissings = 0.05):
        super(ImputaciónMissingsEstimator, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, porcentajeMissings = 0.05):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def deteccion_missings(self, df):
        porcentaje_missings = self.getPorcentajeMissings()
        columnas_missings_imputar = []
        columnas_missings_eliminar = []
        df_size=df.count()
        # Lo hacemos para todas las columnas del Dataframe porque 
        # spués trabajaremos también con variables numéricas
        for c in df.columns:
            missing_porcentaje = df.select([(p.count(p.when(p.isnan(c) | p.col(c).isNull(), c))/df_size)
                                            .alias(c)]).collect()[0][0]
            if missing_porcentaje > porcentaje_missings:
                columnas_missings_eliminar.append(c)
            elif missing_porcentaje != 0:
                columnas_missings_imputar.append(c)
        return columnas_missings_imputar, columnas_missings_eliminar
    
    def calcular_moda(self, df):
        df_categoricas = df.select(*[elem[0] for elem in df.dtypes if elem[1].startswith('string')])
        columnas_categoricas = df_categoricas.columns
        lista_modas = []
        for columna in columnas_categoricas:
            contador = df_categoricas.groupby(columna).count().sort(p.col("count").desc())
            contador.na.drop()
            moda = contador.first()[0]
            lista_modas.append(moda)
        return lista_modas
    
    def _fit(self, df):
        columnas_missings_imputar, columnas_missings_eliminar = self.deteccion_missings(df)
        lista_modas = self.calcular_moda(df)
        
        return ImputacionMissingsModel(columnasMissingsImputar=columnas_missings_imputar,
                                        columnasMissingsEliminar=columnas_missings_eliminar,
                                       listaModas=lista_modas)

# Clase que define el transformer
class ImputacionMissingsModel(Model, HasColumnasMissingsImputar, HasColumnasMissingsEliminar, HasListaModas,  
                              DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, columnasMissingsImputar=[], columnasMissingsEliminar=[], 
              listaModas="",):
        super(ImputacionMissingsModel, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, columnasMissingsImputar=[], columnasMissingsEliminar=[], listaModas=""):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, df):
        columnas_missings_eliminar = self.getColumnasMissingsEliminar()
        columnas_missings_imputar = self.getColumnasMissingsImputar()
        lista_modas = self.getListaModas()
        for i in columnas_missings_eliminar:
            df = df.drop(i)
        diccionario_imputacion = dict(zip(columnas_missings_imputar, lista_modas))
        df = df.fillna(diccionario_imputacion)
        return df

In [6]:
# Ponemos un porcentaje alto para que se realice la imputación
missing_imputacion = ImputaciónMissingsEstimator(porcentajeMissings=0.50)
modelo = Pipeline(stages=[missing_imputacion]).fit(df)
resultado = modelo.transform(df)
resultado.show()

+----------+----------+----------+
|Variable_1|Variable_2|Variable_3|
+----------+----------+----------+
|      Gato|     Perro|     Perro|
|     Perro|     Perro|     Perro|
|     Perro|    Conejo|     Perro|
|      Gato|    Conejo|     Perro|
|    Conejo|     Perro|     Perro|
+----------+----------+----------+

